In [7]:
import pandas as pd
import os
import requests
import glob


In [44]:
#!/usr/bin/env python3
"""
Example Python script illustrating the multi-step ICA pipeline described in your excerpt.

Required libraries:
    numpy, scipy, scikit-learn, nibabel (for loading NIfTI, if needed), etc.
    
DISCLAIMER:
    - This is a simplified illustration only. Adjust as needed for your environment.
    - Actual Infomax ICA and true ICASSO procedures may differ in details from FastICA.
"""

import os
import numpy as np
import nibabel as nib
from sklearn.decomposition import PCA, FastICA
from scipy.stats import skew

# ---------------------------------------------------------------------------
# Step 1: Load and prepare data
# ---------------------------------------------------------------------------
def load_fmri_data(list_of_nifti_paths):
    """
    Example loader that:
      - Reads each preprocessed 4D fMRI file (time x x_dim x y_dim x z_dim).
      - Reshapes to a 2D array: (time, n_voxels).
      - Returns a list of subject data arrays.
    """
    subject_data = []
    for fpath in list_of_nifti_paths:
        img = nib.load(fpath)
        data_4d = img.get_fdata()  # shape: (x_dim, y_dim, z_dim, time)
        # Move time to axis=0 and flatten the spatial dims:
        data_2d = np.reshape(np.moveaxis(data_4d, -1, 0),
                             (data_4d.shape[-1], -1))
        subject_data.append(data_2d)
    return subject_data


def individual_subject_pca(subject_data, n_components=110):
    """
    Perform PCA on each subject’s data (time x voxel) to reduce to `n_components`.
    Return a list of reduced 2D arrays: (time, n_components).
    """
    subject_pcs = []
    for data_2d in subject_data:
        pca = PCA(n_components=n_components)
        reduced = pca.fit_transform(data_2d)  # shape: (time_points, n_components)
        subject_pcs.append(reduced)
    return subject_pcs


# ---------------------------------------------------------------------------
# Step 2 & 3: Concatenate individual PCs and run group-level PCA, then ICA
# ---------------------------------------------------------------------------
def run_group_pca_then_ica(subject_pcs, n_group_components=100,
                           n_ica_runs=100, random_state=0):
    """
    - Concatenate each subject's PCA results along the row dimension (time),
      forming a large group matrix: (sum_of_times, 110).
    - Run a second PCA to reduce to `n_group_components` (default=100).
    - Then run multiple ICA (FastICA) attempts for ICASSO-like approach.
    - Pick best run based on similarity to a "consensus" or by max neg-entropy, etc.
    - Return the best-run's IC mixing matrix and the final group-level ICs.
    """
    # 1) Concatenate:
    # subject_pcs is a list of arrays shape (time, 110)
    group_data = np.concatenate(subject_pcs, axis=0)  # (sum_of_times, 110)
    
    # 2) PCA to reduce to 100 group-level PCs
    group_pca = PCA(n_components=n_group_components, random_state=random_state)
    group_pcs_data = group_pca.fit_transform(group_data)  # shape: (sum_of_times, 100)
    
    # For an "ICASSO-like" approach, run multiple ICA with different random seeds:
    ica_components_list = []
    
    # Typically, you might store all unmixing matrices & then do a clustering step.
    # Here we do a simplified approach, then pick the "best" by average kurtosis, e.g.
    for run_idx in range(n_ica_runs):
        rng = np.random.RandomState(run_idx)  # or vary seeds
        ica_model = FastICA(n_components=n_group_components,
                            random_state=rng,
                            max_iter=1000)
        S_ = ica_model.fit_transform(group_pcs_data)  # shape: (time_points, 100)
        # The columns of `S_` are the estimated IC time-courses (component signals).
        # The mixing matrix W^-1 is ica_model.mixing_. 
        # The actual spatial maps can be derived from S_ or from the pseudo-inverse as needed.
        # We'll store the "spatial" IC patterns as S_.T or do a pinv if you prefer that orientation.
        ica_components_list.append(S_.T)
    
    # Decide which run is "best" – e.g., pick run with highest average neg-entropy or kurtosis:
    # (Below is a crude example using the sum of absolute kurtosis across all components.)
    best_run_idx = None
    best_run_metric = -np.inf
    for i, comps_2d in enumerate(ica_components_list):
        # `comps_2d` shape: (n_components, time_points)
        # we could use kurtosis, or negentropy approximation, etc.
        # for simplicity, we do:
        kurt_vals = np.array([skew(c, bias=False) for c in comps_2d])
        # sum of absolute skew:
        metric = np.sum(np.abs(kurt_vals))
        if metric > best_run_metric:
            best_run_metric = metric
            best_run_idx = i
    
    best_ica_maps = ica_components_list[best_run_idx]  # shape: (100, time_points)
    
    # Reshape or invert to get group-level "spatial" components. 
    # For Infomax-like ICA on PCA outputs, we typically interpret components 
    # by (pseudo-inverse of mixing) or the dot product with group PCA loadings.
    # This example just returns best_ica_maps as "group-level ICs" to be refined if needed.
    return best_ica_maps  # shape: (n_components, time_points)


# ---------------------------------------------------------------------------
# Step 4: Flip IC if its skewness is negative
# ---------------------------------------------------------------------------
def flip_negative_skew(ics_2d):
    """
    Given 2D array of shape (n_components, n_timepoints/voxels),
    compute skewness for each row, flip if negative.
    Returns the array with flips applied.
    """
    flipped_ics = ics_2d.copy()
    n_comp = flipped_ics.shape[0]
    for i in range(n_comp):
        # compute skewness for the ith row
        val = skew(flipped_ics[i, :], bias=False)
        if val < 0:
            flipped_ics[i, :] *= -1
    return flipped_ics


# ---------------------------------------------------------------------------
# Step 5: Greedy matching of two sets of IC maps
# ---------------------------------------------------------------------------
def greedy_spatial_match(ics_a, ics_b, corr_threshold=0.4):
    """
    ics_a, ics_b: each shape = (n_components, n_voxels) or (n_components, n_timepoints)
                  but typically you'd have them in "spatial map" form (component x voxel).
    
    1. Compute an abs-correlation matrix between the two sets of ics, shape= (Na, Nb).
    2. Repeatedly pick the max correlation pair, sign-flip if original correlation is negative.
    3. Zero out that row and column to exclude them from further pairing.
    4. Return the matched pairs that exceed the threshold, plus their sign-flipped versions.
    """
    nA, nV = ics_a.shape
    nB, _ = ics_b.shape
    # correlation matrix (absolute value)
    # We'll keep track of the sign as well. 
    corr_mat = np.zeros((nA, nB))
    sign_mat = np.zeros((nA, nB))
    
    for i in range(nA):
        for j in range(nB):
            corr_ij = np.corrcoef(ics_a[i,:], ics_b[j,:])[0,1]
            corr_mat[i,j] = abs(corr_ij)
            sign_mat[i,j] = np.sign(corr_ij)  # +1 or -1 or 0
    
    # Now do the iterative "greedy" selection:
    matched_pairs = []  # will store tuples like (idxA, idxB, correlation, sign_of_correlation)
    
    # Make a copy of corr_mat to zero out as we pick pairs
    tmp_corr = corr_mat.copy()
    
    for _ in range(nA):  # up to min(nA,nB) matches
        # find max in tmp_corr
        i_max, j_max = np.unravel_index(np.argmax(tmp_corr), tmp_corr.shape)
        max_val = tmp_corr[i_max, j_max]
        if max_val < corr_threshold:
            # no more pairs exceed threshold
            break
        
        matched_pairs.append((i_max, j_max, max_val, sign_mat[i_max,j_max]))
        
        # zero out that row and column
        tmp_corr[i_max, :] = 0.0
        tmp_corr[:, j_max] = 0.0
    
    # If correlation sign was negative, we sign-flip ICS_B’s component 
    # or ICS_A’s, but typically flip ICS_B for convenience. 
    # (In actual pipeline, you might want to store a separate copy for the flips.)
    # We'll do it in-place here for demonstration.
    for (ia, ib, val, sgn) in matched_pairs:
        if sgn < 0:
            ics_b[ib,:] *= -1
    
    # Return the subset of matched pairs that exceed threshold 
    # plus the possibly sign-flipped ics_b:
    final_pairs = [p for p in matched_pairs if p[2] >= corr_threshold]
    return final_pairs, ics_a, ics_b


In [3]:
# ---------------------------------------------------------------------------
# Main demonstration function
# ---------------------------------------------------------------------------
def main_example():
    """
    Demonstrate the pipeline, step by step, using hypothetical file lists
    for two cohorts: controls and disease.
    """
    # Hypothetical example: lists of NIfTI paths
    #  - Replace with your actual preprocessed fMRI NIfTI files
    control_paths = ["/path/to/control_subject_01_preproc.nii.gz",
                     "/path/to/control_subject_02_preproc.nii.gz",
                     # ...
                    ]
    disease_paths = ["/path/to/disease_subject_01_preproc.nii.gz",
                     "/path/to/disease_subject_02_preproc.nii.gz",
                     # ...
                    ]
    
    # 1) Load data & do subject-level PCA
    control_data = load_fmri_data(control_paths)   # list of arrays (time x voxel)
    disease_data = load_fmri_data(disease_paths)   # list of arrays (time x voxel)
    
    control_pcs = individual_subject_pca(control_data, n_components=110)
    disease_pcs = individual_subject_pca(disease_data, n_components=110)
    
    # 2 & 3) Group-level PCA then ICA with repeated runs (ICASSO style)
    ctrl_group_ics = run_group_pca_then_ica(control_pcs,
                                            n_group_components=100,
                                            n_ica_runs=100,
                                            random_state=0)
    dis_group_ics  = run_group_pca_then_ica(disease_pcs,
                                            n_group_components=100,
                                            n_ica_runs=100,
                                            random_state=0)
    # ctrl_group_ics, dis_group_ics each shape: (100, group_time_points)
    # but for matching we usually want them as (100, voxel), i.e. "spatial maps."
    # If your group-time dimension is not the same as voxel dimension, you'd 
    # typically invert the mixing or do post-processing to obtain spatial maps.
    # We'll pretend these are already "spatial" for the sake of demonstration.
    
    # 4) Flip negative skewness
    ctrl_group_ics = flip_negative_skew(ctrl_group_ics)
    dis_group_ics  = flip_negative_skew(dis_group_ics)
    
    # 5) Greedy matching between the two sets
    matched_pairs, ctrl_flipped, dis_flipped = greedy_spatial_match(
        ctrl_group_ics,
        dis_group_ics,
        corr_threshold=0.4
    )
    
    print(f"Number of matched IC pairs (corr>0.4): {len(matched_pairs)}")
    for pair in matched_pairs:
        iA, iB, corr_val, sign_ = pair
        print(f"  Pair: IC_ctrl={iA}, IC_dis={iB}, corr={corr_val:.3f}, sign={sign_}")
    
    # The matched_pairs with correlation > 0.4 are considered reproducible ICs. 
    # Next steps might include:
    #   - Inspect each matched IC pair’s spatial pattern.
    #   - Exclude artifactual components using heuristics (peak in gray matter, etc.).
    #   - Compare final sets of reproducible ICNs across cohorts.
    #   - Downstream connectivity/functional analyses.

    print("Done. This demonstration performed the group-ICA-like pipeline.")


In [4]:
metadata_df = pd.read_csv("./Phenotypic_V1_0b_preprocessed1.csv")

In [18]:
metadata_df[metadata_df["DX_GROUP"] == 1].head()

,Unnamed: 0.1,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [21]:
metadata_df[metadata_df["DX_GROUP"] == 2].head()

,Unnamed: 0.1,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
26,26,27,50030,27,50030,PITT,Pitt_0050030,2,0,25.12,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,1
27,27,28,50031,28,50031,PITT,Pitt_0050031,2,0,12.92,...,NaN,OK,NaN,maybe,ic-cerebellum_temporal_lobe,OK,NaN,OK,NaN,1
28,28,29,50032,29,50032,PITT,Pitt_0050032,2,0,19.80,...,NaN,OK,NaN,maybe,ic-cerebellum_temporal_lobe,OK,NaN,OK,NaN,1
29,29,30,50033,30,50033,PITT,Pitt_0050033,2,0,12.15,...,NaN,OK,NaN,maybe,ic-cerebellum_temporal_lobe,OK,NaN,OK,NaN,1
30,30,31,50034,31,50034,PITT,Pitt_0050034,2,0,14.77,...,NaN,OK,NaN,maybe,ic-cerebellum_temporal_lobe,OK,NaN,OK,NaN,1


In [34]:
control_file_IDS = metadata_df[metadata_df["DX_GROUP"] == 2]["FILE_ID"].to_list()
ASD_file_IDS = metadata_df[metadata_df["DX_GROUP"] == 1]["FILE_ID"].to_list()

In [35]:
all_fmri_files = glob.glob("/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/ABIDE_MNI_SMOOTH/*")

In [42]:
control_paths = [path for path in all_fmri_files if os.path.basename(path).replace("_MNI_smoothed.nii.gz", "") in control_file_IDS]
disease_paths = [path for path in all_fmri_files if os.path.basename(path).replace("_MNI_smoothed.nii.gz", "") in ASD_file_IDS]

In [43]:
# Hypothetical example: lists of NIfTI paths
#  - Replace with your actual preprocessed fMRI NIfTI files

# 1) Load data & do subject-level PCA
control_data = load_fmri_data(control_paths[:25])   # list of arrays (time x voxel)
disease_data = load_fmri_data(disease_paths[:25])   # list of arrays (time x voxel)

control_pcs = individual_subject_pca(control_data, n_components=110)
disease_pcs = individual_subject_pca(disease_data, n_components=110)



InvalidParameterError: The 'whiten' parameter of FastICA must be a str among {'unit-variance', 'arbitrary-variance'} or a bool among {False}. Got True instead.

In [45]:
# 2 & 3) Group-level PCA then ICA with repeated runs (ICASSO style)
ctrl_group_ics = run_group_pca_then_ica(control_pcs,
                                        n_group_components=100,
                                        n_ica_runs=100,
                                        random_state=0)
dis_group_ics  = run_group_pca_then_ica(disease_pcs,
                                        n_group_components=100,
                                        n_ica_runs=100,
                                        random_state=0)
# ctrl_group_ics, dis_group_ics each shape: (100, group_time_points)
# but for matching we usually want them as (100, voxel), i.e. "spatial maps."
# If your group-time dimension is not the same as voxel dimension, you'd 
# typically invert the mixing or do post-processing to obtain spatial maps.
# We'll pretend these are already "spatial" for the sake of demonstration.

# 4) Flip negative skewness
ctrl_group_ics = flip_negative_skew(ctrl_group_ics)
dis_group_ics  = flip_negative_skew(dis_group_ics)



ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4452 and the array at index 1 has size 5436

In [ ]:
# 5) Greedy matching between the two sets
matched_pairs, ctrl_flipped, dis_flipped = greedy_spatial_match(
    ctrl_group_ics,
    dis_group_ics,
    corr_threshold=0.4
)

print(f"Number of matched IC pairs (corr>0.4): {len(matched_pairs)}")
for pair in matched_pairs:
    iA, iB, corr_val, sign_ = pair
    print(f"  Pair: IC_ctrl={iA}, IC_dis={iB}, corr={corr_val:.3f}, sign={sign_}")

# The matched_pairs with correlation > 0.4 are considered reproducible ICs. 
# Next steps might include:
#   - Inspect each matched IC pair’s spatial pattern.
#   - Exclude artifactual components using heuristics (peak in gray matter, etc.).
#   - Compare final sets of reproducible ICNs across cohorts.
#   - Downstream connectivity/functional analyses.

print("Done. This demonstration performed the group-ICA-like pipeline.")


In [46]:
ctrl_group_ics

array([[-8.49688443e-03, -1.07463879e-01,  2.18822596e-01, ...,
         1.43537749e-01, -3.08436589e-01, -7.16405206e-01],
       [ 4.51096422e-02,  3.44334725e-02, -1.58252708e-02, ...,
         6.88624111e-02,  1.50578049e-01,  1.98669039e-01],
       [-4.64069301e-01, -6.10793096e-01, -2.89490354e-01, ...,
        -7.00814036e-01,  2.49008887e-01,  7.74731790e-01],
       ...,
       [-8.77152491e-01, -9.55246325e-01,  1.54314870e-01, ...,
         9.43521484e-01, -7.79410708e-02, -8.60677198e-01],
       [-1.25323515e-03,  1.69680524e-02,  1.35392343e-02, ...,
         3.09528369e+00,  9.81202538e-01, -1.70429622e+00],
       [-1.51925821e+00, -1.19345454e+00,  6.97358972e-01, ...,
        -1.72784682e+00, -2.35007360e+00, -2.09667513e+00]],
      shape=(100, 4452))

In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.stats import zscore
from scipy.signal import convolve

def adaptive_ica_single_component(Xw, template, alpha=0.5, max_iter=200, tol=1e-5):
    """
    Demonstration of a single-component GIG-ICA-like update.
    Xw:   (T, V) the subject's *whitened* data (time x voxel).
    template: (V,) the l-th template (already normalized or not).
    alpha: weighting (0.5 => half independence, half prior similarity).
    max_iter, tol: iteration settings.
    
    Returns:
        w:  (V,) the unmixing vector for this single component
        s:  (T,) the subject-specific time course (or can interpret it as the comp. activation).
        c:  (V,) the subject-specific spatial map S^l_k
    NOTE:
      - This is a conceptual gradient-based approach. Real GIG-ICA uses Infomax-like updates.
      - We treat 'negentropy' term with a popular approximation via e.g. G(u)=log(cosh(u)).
      - The prior-similarity term is a dot product with the template (or correlation).
    """
    T, V = Xw.shape
    # Initialize w randomly, then normalize
    rng = np.random.RandomState(0)
    w = rng.randn(V)
    w /= norm(w)

    # Precompute a normalized template if desired:
    # If we want a pure dot product measure: just do template_norm = template.
    # If we prefer correlation-like measure, standardize template:
    ttemp = zscore(template)  # zero-mean, unit-std

    def negentropy_grad(s, Xw):
        """
        Approx gradient of negentropy wrt w.
        We'll use G(u)= log(cosh(u)) as a standard nonlinearity.
        s = Xw @ w^T => shape (T,)
        """
        # derivative of logcosh(s) ~ tanh(s)
        # gradient wrt w ~ Xw.T * tanh(s)
        return Xw.T.dot(np.tanh(s))

    def similarity_grad(s, Xw, ttemp):
        """
        Approx gradient of correlation wrt w. 
        correlation(s, template) ~ (s - mean(s))*(template - mean(template)) / (std(s)*std(template))
        For simplicity, we skip some partial derivative complexities and treat it as a dot product
        with zscored data to illustrate the idea. 
        """
        # s is shape (T,). Let's zscore s => sZ. Then dot(sZ, ttemp) is correlation ~ sZ dot ttemp
        sZ = zscore(s)  # zero-mean, unit-std
        # derivative wrt w => derivative of sZ wrt s times derivative of s wrt w...
        # but let's keep it simpler: approximate
        # We'll do a direct dot-product measure: "similarity ~ s^T * template"
        # Then gradient is Xw.T * template
        # for correlation-like measure, we can do Xw.T * ttemp
        grad_approx = Xw.T.dot(ttemp)
        return grad_approx

    # Iterative gradient-based update
    last_w = w.copy()
    for i in range(max_iter):
        # Project: s = Xw @ w
        s = Xw.dot(w)
        # Weighted gradient = alpha*(negentropy_grad) + (1-alpha)*(similarity_grad)
        g1 = negentropy_grad(s, Xw)
        g2 = similarity_grad(s, Xw, ttemp)
        grad = alpha*g1 + (1-alpha)*g2
        
        # Orthogonalize if you want multiple comps. We'll do single comp for demonstration.
        
        # Update step: w_new = old + step_size * grad
        # step_size can be adapted. We'll pick a small constant. 
        step_size = 1e-5  
        w_new = w + step_size * grad
        
        # re-normalize
        w_new /= norm(w_new)
        
        # check convergence
        if norm(w_new - w) < tol:
            w = w_new
            break
        
        w = w_new
    
    # final projection
    s_final = Xw.dot(w)  # shape (T,)
    # subject-specific SPATIAL map c = Xw^T * s, or see how you interpret it
    # typically if w is the unmixing, then c = w^T * (cov or something)
    # We'll do a direct map:
    c_final = s_final @ Xw / (T - 1)  # rough approach

    # Z-score the spatial map if desired
    c_final_z = zscore(c_final)
    
    return w, s_final, c_final_z


def subject_adaptive_ica(X, group_templates, alpha=0.5):
    """
    Illustrate how to get ALL subject-specific networks 
    given the set of group-level templates.
    
    X:  (T, V) raw data for one subject (time x voxel).
    group_templates: (N, V) for N templates/ICNs.
    alpha: weighting for independence vs. prior similarity.
    
    Steps:
      1) Whiten X
      2) For each template -> run the single-component optimization
      3) Collect subject-specific networks and time-courses
    Returns:
      sps_maps:   array (N, V)
      time_courses: array (N, T)
    """
    T, V = X.shape
    # 1) Whiten the data
    #   Typically we do PCA => whiten => Xw shape (T, V)
    X_mean = X.mean(axis=0)
    Xc = X - X_mean
    # compute SVD or PCA:
    U, S, VT = np.linalg.svd(Xc, full_matrices=False)
    # whitened data
    # Xc = U @ diag(S) @ VT so let's do Xw = U @ I
    # or we can do: Xw = (U * diag(1/S))^T ...
    # let's do the simpler: Xw = U * factor
    # But note shape: we want Xw => (T, V)
    # We'll do Xw = U * sqrt(T-1). But let's keep it simpler:
    Xw = U.copy()
    
    # 2) For each template, do the single-comp solver
    N = group_templates.shape[0]
    sps_maps = np.zeros((N, V))
    time_courses = np.zeros((N, T))
    for l in range(N):
        template_l = group_templates[l,:]
        w_l, s_l, c_l = adaptive_ica_single_component(
            Xw, template_l, alpha=alpha, max_iter=200, tol=1e-5
        )
        # c_l is the subject-specific spatial map
        sps_maps[l, :] = c_l
        
        # Also define time-course as correlation between data and that map,
        # or project raw data onto c_l.  We'll do a simple dot:
        # time-course = Xc * c_l^T
        # but typically you might want to invert the mixing matrix, etc. 
        # For demonstration:
        tc = (Xc.dot(c_l))  # shape (T,)
        # optional zscore:
        tc_z = zscore(tc)
        time_courses[l, :] = tc_z
    
    return sps_maps, time_courses


In [ ]:
import numpy as np
from scipy.stats import pearsonr, zscore

def compute_static_fnc(time_courses):
    """
    Given time_courses of shape (N, T) for N ICNs, T timepoints,
    compute an N x N static functional connectivity matrix 
    by Pearson correlation between each pair of time-courses.
    """
    N, T = time_courses.shape
    fnc_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            r, _ = pearsonr(time_courses[i,:], time_courses[j,:])
            fnc_matrix[i,j] = r
    return fnc_matrix

def compute_dynamic_fnc(time_courses, window_length=30, step=1):
    """
    Simple demonstration of dynamic FNC with a sliding window.
    time_courses: (N, T)
    window_length: number of time points per window
    step: step size (in timepoints) for sliding
    
    Returns:
      A list of FNC matrices, each (N, N), or we can stack them in array (numWindows, N, N).
    """
    N, T = time_courses.shape
    windowed_FNC = []
    start = 0
    while (start + window_length) <= T:
        seg = time_courses[:, start : start+window_length]  # shape (N, window_length)
        # correlation
        fnc_mat = np.zeros((N, N))
        for i in range(N):
            for j in range(N):
                r, _ = pearsonr(seg[i,:], seg[j,:])
                fnc_mat[i,j] = r
        windowed_FNC.append(fnc_mat)
        start += step
    
    # Optionally, convert to a 3D array: (numWindows x N x N)
    windowed_FNC = np.array(windowed_FNC)
    return windowed_FNC


In [ ]:
# Suppose we already obtained for one subject:
#   sps_maps (N, V) and time_courses (N, T).
# Example: compute sFNC
sFNC = compute_static_fnc(time_courses)  # shape (N, N)

# Example: compute dFNC using a 30-TR window sliding by 1
dFNC_array = compute_dynamic_fnc(time_courses, window_length=30, step=1)
# shape: (num_windows, N, N)


In [22]:
raise ValueError()

ValueError: 

In [17]:
data_4d.max()

np.float64(1466.948974609375)

In [12]:
data_2d.shape

(124, 259200)

In [6]:
len(df["SUB_ID"].unique())

1112

In [7]:
df["SITE_ID"].unique()

array(['CALTECH', 'CMU', 'KKI', 'LEUVEN_1', 'LEUVEN_2', 'MAX_MUN', 'NYU',
       'OHSU', 'OLIN', 'PITT', 'SBL', 'SDSU', 'STANFORD', 'TRINITY',
       'UCLA_1', 'UCLA_2', 'UM_1', 'UM_2', 'USM', 'YALE'], dtype=object)

In [8]:
df["DX_GROUP"].unique()

array([1, 2])

In [9]:
df.columns

Index(['SITE_ID', 'SUB_ID', 'FILE_ID', 'DX_GROUP', 'DSM_IV_TR', 'AGE_AT_SCAN',
       'SEX', 'HANDEDNESS_CATEGORY', 'HANDEDNESS_SCORES', 'FIQ', 'VIQ', 'PIQ',
       'FIQ_TEST_TYPE', 'VIQ_TEST_TYPE', 'PIQ_TEST_TYPE',
       'ADI_R_SOCIAL_TOTAL_A', 'ADI_R_VERBAL_TOTAL_BV', 'ADI_RRB_TOTAL_C',
       'ADI_R_ONSET_TOTAL_D', 'ADI_R_RSRCH_RELIABLE', 'ADOS_MODULE',
       'ADOS_TOTAL', 'ADOS_COMM', 'ADOS_SOCIAL', 'ADOS_STEREO_BEHAV',
       'ADOS_RSRCH_RELIABLE', 'ADOS_GOTHAM_SOCAFFECT', 'ADOS_GOTHAM_RRB',
       'ADOS_GOTHAM_TOTAL', 'ADOS_GOTHAM_SEVERITY', 'SRS_VERSION',
       'SRS_RAW_TOTAL', 'SRS_AWARENESS', 'SRS_COGNITION', 'SRS_COMMUNICATION',
       'SRS_MOTIVATION', 'SRS_MANNERISMS', 'SCQ_TOTAL', 'AQ_TOTAL',
       'COMORBIDITY', 'CURRENT_MED_STATUS', 'MEDICATION_NAME',
       'OFF_STIMULANTS_AT_SCAN', 'VINELAND_RECEPTIVE_V_SCALED',
       'VINELAND_EXPRESSIVE_V_SCALED', 'VINELAND_WRITTEN_V_SCALED',
       'VINELAND_COMMUNICATION_STANDARD', 'VINELAND_PERSONAL_V_SCALED',
       'VINELAN

In [17]:
preprocessed_metadata_df = pd.read_csv("/blue/ruogu.fang/ryoi360/projects/fmri_vlm/results/2025_02_25_ABIDE_processing/Phenotypic_V1_0b_preprocessed1.csv")

In [21]:
for col in preprocessed_metadata_df.columns:
    print(col)

Unnamed: 0.1
Unnamed: 0
SUB_ID
X
subject
SITE_ID
FILE_ID
DX_GROUP
DSM_IV_TR
AGE_AT_SCAN
SEX
HANDEDNESS_CATEGORY
HANDEDNESS_SCORES
FIQ
VIQ
PIQ
FIQ_TEST_TYPE
VIQ_TEST_TYPE
PIQ_TEST_TYPE
ADI_R_SOCIAL_TOTAL_A
ADI_R_VERBAL_TOTAL_BV
ADI_RRB_TOTAL_C
ADI_R_ONSET_TOTAL_D
ADI_R_RSRCH_RELIABLE
ADOS_MODULE
ADOS_TOTAL
ADOS_COMM
ADOS_SOCIAL
ADOS_STEREO_BEHAV
ADOS_RSRCH_RELIABLE
ADOS_GOTHAM_SOCAFFECT
ADOS_GOTHAM_RRB
ADOS_GOTHAM_TOTAL
ADOS_GOTHAM_SEVERITY
SRS_VERSION
SRS_RAW_TOTAL
SRS_AWARENESS
SRS_COGNITION
SRS_COMMUNICATION
SRS_MOTIVATION
SRS_MANNERISMS
SCQ_TOTAL
AQ_TOTAL
COMORBIDITY
CURRENT_MED_STATUS
MEDICATION_NAME
OFF_STIMULANTS_AT_SCAN
VINELAND_RECEPTIVE_V_SCALED
VINELAND_EXPRESSIVE_V_SCALED
VINELAND_WRITTEN_V_SCALED
VINELAND_COMMUNICATION_STANDARD
VINELAND_PERSONAL_V_SCALED
VINELAND_DOMESTIC_V_SCALED
VINELAND_COMMUNITY_V_SCALED
VINELAND_DAILYLVNG_STANDARD
VINELAND_INTERPERSONAL_V_SCALED
VINELAND_PLAY_V_SCALED
VINELAND_COPING_V_SCALED
VINELAND_SOCIAL_STANDARD
VINELAND_SUM_SCORES
VINELAND_ABC_ST

In [26]:
motion_filtered_df = preprocessed_metadata_df[(preprocessed_metadata_df['func_mean_fd'] <= 0.2) & (preprocessed_metadata_df['func_num_fd'] < 20)]

print(f"Subjects after stricter filtering: {len(motion_filtered_df)}")

Subjects after stricter filtering: 714


In [22]:
len(preprocessed_metadata_df.columns)

106

In [19]:
preprocessed_metadata_df["func_mean_fd"]

0       0.116828
1       0.322092
2       0.127745
3       0.128136
4       0.070143
          ...   
1107    0.116186
1108    0.140171
1109    0.154887
1110    0.048246
1111    0.168913
Name: func_mean_fd, Length: 1112, dtype: float64

In [28]:
preprocessed_metadata_df["func_fwhm"].max()

np.float64(3.7534808758)

In [10]:
import glob

In [11]:
abide_files = glob.glob("/orange/ruogu.fang/ryoi360/ABIDE/*")

In [12]:
len(abide_files)

1035

In [11]:
import numpy as np
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt

In [13]:
fmri_img = nib.load(abide_files[0])
fmri_data = fmri_img.get_fdata()

In [14]:
fmri_data.shape

(61, 73, 61, 116)

In [15]:
# Get the voxel size from the affine transformation matrix
voxel_size = np.sqrt(np.sum(fmri_img.affine[:3, :3] ** 2, axis=0))
print("Original Voxel Size (mm):", voxel_size)

Original Voxel Size (mm): [3. 3. 3.]


In [ ]:
1. Install Required Packages
If you haven't installed Nipype, Nibabel, and NiLearn, do so using:

bash
Copy
Edit
pip install nipype nibabel nilearn numpy scipy


In [ ]:
2. Preprocessing Steps in Python
Step 1: Rigid Body Motion Correction
Use SPM's Realign function via Nipype.
Alternatively, use FSL's MCFLIRT.
SPM12 (via Nipype)
python
Copy
Edit
from nipype.interfaces.spm import Realign

realign = Realign()
realign.inputs.in_files = 'subject_func.nii'  # Replace with your file path
realign.inputs.register_to_mean = True
realign.run()
FSL Alternative
python
Copy
Edit
from nipype.interfaces.fsl import MCFLIRT

mcflirt = MCFLIRT()
mcflirt.inputs.in_file = 'subject_func.nii'
mcflirt.inputs.out_file = 'motion_corrected.nii'
mcflirt.run()
Step 2: Slice Timing Correction
Adjusts for differences in slice acquisition time.
Requires TR (repetition time) and slice order.
python
Copy
Edit
from nipype.interfaces.spm import SliceTiming

slice_timing = SliceTiming()
slice_timing.inputs.in_files = 'motion_corrected.nii'
slice_timing.inputs.time_repetition = 2.0  # Set the correct TR
slice_timing.run()
Step 3: Normalization to MNI Space
Warp the functional data into MNI152 template.
Use SPM's Normalize or FSL's FLIRT/FNIRT.
SPM Normalization
python
Copy
Edit
from nipype.interfaces.spm import Normalize12

normalize = Normalize12()
normalize.inputs.image_to_align = 'slice_time_corrected.nii'
normalize.inputs.apply_to_files = ['slice_time_corrected.nii']
normalize.inputs.jobtype = 'estwrite'  # Estimate and apply transformation
normalize.run()
FSL FLIRT Alternative
python
Copy
Edit
from nipype.interfaces.fsl import FLIRT

flirt = FLIRT()
flirt.inputs.in_file = 'slice_time_corrected.nii'
flirt.inputs.reference = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
flirt.inputs.out_file = 'normalized.nii'
flirt.run()
Step 4: Resampling to 3×3×3 mm³
Use NiLearn for resampling.
python
Copy
Edit
from nilearn.image import resample_img
import nibabel as nib

img = nib.load("normalized.nii")

resampled_img = resample_img(img, target_affine=np.diag([3, 3, 3, 1]))
nib.save(resampled_img, "resampled_3mm.nii")
Step 5: Spatial Smoothing (FWHM = 6 mm)
Apply Gaussian smoothing using NiLearn.
python
Copy
Edit
from nilearn.image import smooth_img

smoothed_img = smooth_img("resampled_3mm.nii", fwhm=6)
smoothed_img.to_filename("smoothed.nii")
